In [ ]:
import numpy as np
from scipy.spatial import ConvexHull
from scipy.optimize import minimize

def detect_main_direction(points):
    # 计算多边形的主要轴
    centroid = np.mean(points, axis=0)
    _, eigen_vectors = np.linalg.eig(np.cov(points.T))
    main_direction = eigen_vectors[:, np.argmax(np.abs(eigen_vectors.sum(axis=0)))]
    return main_direction

def simplify_polygon(points, tolerance):
    # 使用道格拉斯-普克算法减少顶点数量
    d = np.abs(np.dot(points - points[0], np.roll(np.array([1, -1]), -1)))
    mask = np.concatenate([[True], d > tolerance])
    simplified_points = points[mask]
    return simplified_points

def regularize_polygon(points, tolerance=0.1):
    # 计算凸包
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]

    # 检测主要方向
    main_direction = detect_main_direction(hull_points)

    # 定义目标函数，计算顶点之间的夹角与直角之间的差异
    def objective(x):
        angles = np.arctan2(x[1::2] - x[:-1:2], np.roll(x[1::2] - x[:-1:2], -1))
        return np.sum(np.abs(np.pi/2 - angles))

    # 定义约束条件，确保顶点位置在凸包边界内
    def constraint(x):
        return hull.equations.dot(np.hstack([x[0::2], x[1::2], 1])) - 1e-6

    # 初始化优化问题
    x0 = hull_points.flatten()
    bounds = [(min(hull_points[:, 0]), max(hull_points[:, 0]))] * len(hull_points) + [(min(hull_points[:, 1]), max(hull_points[:, 1]))] * len(hull_points)
    constraints = {'type': 'ineq', 'fun': constraint}

    # 调用优化方法，迭代调整顶点位置
    result = minimize(objective, x0, bounds=bounds, constraints=constraints)
    optimized_points = result.x.reshape((-1, 2))

    # 减少顶点数量
    simplified_points = simplify_polygon(optimized_points, tolerance)

    return simplified_points

# 复杂测试用例
points = np.array([(0, 0), (1, 1), (3, 0), (4, 2), (6, 1), (7, 3), (8, 2), (9, 0), (10, 2), (11, 1), (12, 3), (13, 2), (14, 0)])
optimized_points = regularize_polygon(points)
print("原始多边形顶点：", points)
print("规则化后的多边形顶点：", optimized_points)